In [114]:
import pandas as pd
from pathlib import Path


### Level 1:
### Block of solving and studying pandas core

In [ ]:
class Solution:

    def __init__(self, input_file: str) -> None:
        self.root_dir = Path.cwd().parent
        self.file_dir = self.root_dir / "data" / input_file
        self.df = pd.read_csv(self.file_dir)

    def _create_path(self, folder_name: str, file_name: str) -> Path:
        folder_dir = self.root_dir / folder_name
        folder_dir.mkdir(exist_ok=True)
        file_dir = folder_dir / Path(file_name)
        return file_dir

    # Filter only rows where duration_min > 25, 
    # sort this rows in descending order,
    # save result into solutions/L1_filtered.xlsx
    def L1_task_1(self, file_name: str) -> None:
        result = self.df[self.df["duration_min"] > 25]
        result = result.sort_values("weight_kg", ascending=False)
        file_dir = self._create_path("solutions", file_name)
        result.to_excel(file_dir, index=False)
    
    # Group all trainings by cities 
    # for each group find average weight, total repeat, amount of unique users
    # Save result into solutions/L1_summary.xlsx
    def L1_task_2(self, file_name: str) -> None:
        grouped_by_cities = self.df.groupby("city").agg({
            "weight_kg": "mean",
            "reps": "sum",
            "user_id": "nunique"
        })
        file_dir = self._create_path("solutions", file_name)
        grouped_by_cities.to_excel(file_dir, index=False)

    # Filter rows where body_part != "cardio"
    # group by body_part and display:
    #   average weight and average reps
    # Save in solutions/L1_report.xlsx
    def L1_task_3(self, file_name: str):
        no_cardio = self.df[self.df["body_part"] != "cardio"]
        grouped_by_body_part = no_cardio.groupby("body_part").agg({
            "weight_kg": "mean",
            "reps": "mean"
        })
        file_dir = self._create_path("solutions", file_name)
        grouped_by_body_part.to_excel(file_dir, index=False)
        

In [121]:
my_var = Solution("workouts.csv")
my_var.L1_task_1("L1_filtered.xlsx")

In [122]:
my_var = Solution("workouts.csv")
my_var.L1_task_2("L1_grouped.xlsx")

In [123]:
my_var = Solution("workouts.csv")
my_var.L1_task_3("L1_summary.xlsx")

### Level 2:
### Advanced pandas methods for to manipulate tables

In [253]:
class Solution_2:
    
    def __init__(self, file_name: str, dim_name: str) -> None:
        self.root_dir = Path.cwd().parent
        self.file_dir = self.root_dir / "data" / Path(file_name)
        self.dim_dir = self.root_dir / "data" / Path(dim_name)
        self.df = pd.read_csv(self.file_dir)
        self.dim = pd.read_csv(self.dim_dir)
    
    def _path_constructor(self, folder_name: str, file_name: str) -> Path:
        folder_dir = self.root_dir / folder_name
        folder_dir.mkdir(exist_ok=True)
        file_dir = folder_dir / file_name
        return file_dir

    @staticmethod     
    def _weight_compare(row: pd.Series):
        return "Above / equal average" if row["weight_kg"] >= row["avg_weight_city"] else "Below average"
    
    # Fix "dim" file by removing invalid rows, 
    # merge my file and dim by city, with validating it in "m:1" sequence, with indicator
    # after merging if there are cities which appear only in left table, return them and their amount
    # replace them in table with "Unknown"
    # final result save as "solutions/L2_joined.xlsx"
    def L2_task_1(self, file_name: str, invalid_file: str):
        self.dim["country"] = self.dim["country"].astype("string")
        invalidity_mask = self.dim["country"].isna() | (self.dim["country"].str.strip() == "")
        invalid_rows = self.dim[invalidity_mask]
        invalid_rows.to_excel(self._path_constructor("invalid_datas", invalid_file))
        valid_dim = self.dim[~invalidity_mask]
        merged_df = self.df.merge(valid_dim,
                                  how="left",
                                  on="city",
                                  suffixes=("_left", "_right"),
                                  validate="m:1",
                                  indicator=True)
        
        invalid_cities_mask = merged_df["_merge"] == "left_only"
        invalid_cities_amount = merged_df[invalid_cities_mask].shape[0]
        merged_df.loc[invalid_cities_mask, "city"] = "Unknown"
        right_cols = [c for c in valid_dim.columns if c != "city"]
        merged_df = merged_df.drop(columns=right_cols, errors="ignore")
        merged_df = merged_df.merge(valid_dim,
                                  how="left",
                                  on="city",
                                  suffixes=("_left", "_right"),
                                  validate="m:1",
                                  indicator="my_col")
        file_dir = self._path_constructor("solutions", "L2_joined.xlsx")
        merged_df.to_excel(file_dir, index=False)
    
    # Challenge to change Wide format - Разобрать, нихуя не понял
    # def L2_task_2(self) -> pd.DataFrame:
    #     self.df.melt(id_vars=["user_id"], var_name=)


    # Add new column avg_weight_city, found for each city
    # compare for each row if its own value is higher then average for city
    # save new result with new column to solutions/L2_transform.xlsx
    def L2_task_3(self) -> None:
        try:
            self.df["weight_kg"] = self.df["weight_kg"].astype(float)
        except ValueError:
            raise  ValueError("All values in column 'weight_kg' have to be convertable to numeric values")
        
        self.df["avg_weight_city"] = self.df.groupby("city")["weight_kg"].transform("mean")
        self.df["compared_to_avg"] = self.df.apply(self._weight_compare, axis=1)
        file_dir = self._path_constructor("solutions", "L2_transform.xlsx")
        self.df.to_excel(file_dir)

    
    # Take column body_part and modify it by using pd.Categorical
    # Make my own order for categories for ex.: ["cardio", "legs", "back", "chest"]
    # Sort dataset by this order
    # save result into solutions/L2_categorical.xlsx
    def L2_task_4(self) -> None:
        self.df["body_part"] = pd.Categorical(self.df["body_part"], categories=["chest", "biceps", "back", "abs", "shoulders", "legs", "cardio"], ordered=True)
        self.df = self.df.sort_values("body_part")
        file_dir = self._path_constructor("solutions", "L2_categorical.xlsx")
        self.df.to_excel(file_dir, index=False)
        
    # Find rows with same value in (user_id, exercise_name)
    # Delete duplicates, and remain 1st appereance of this 
    # save fixed dataset in solutions/L2_deduplicated.xlsx
    def L2_task_5(self) -> None:
        self.df = self.df.drop_duplicates(subset=["user_id", "exercise"])
        file_dir = self._path_constructor("solutions", "L2_deduplicated.xlsx")
        self.df.to_excel(file_dir, index=False)

    # transform all date values into datetime objects 
    # Create separate columns for year, month, day 
    # Save result into solutions/L2_dates.xlsx
    def L2_task_6(self):
        self.df["date"] = pd.to_datetime(self.df["date"], errors="coerce", format="%Y-%m-%d")
        self.df["year"] = self.df["date"].dt.year
        self.df["month"] = self.df["date"].dt.month
        self.df["day"] = self.df["date"].dt.day
        return self.df

In [254]:
my_var_2 = Solution_2("workouts.csv", "cities_lookup.csv")
my_var_2.L2_task_1("L2_summary.xlsx", "L2_task_1_inv_data.xlsx")

In [256]:
my_var_2 = Solution_2("workouts.csv", "cities_lookup.csv")
my_var_2.L2_task_3()

In [219]:
my_var_2 = Solution_2("workouts.csv", "cities_lookup.csv")
my_var_2.L2_task_4()

In [229]:
my_var_2 = Solution_2("workouts.csv", "cities_lookup.csv")
my_var_2.L2_task_5()

In [255]:
my_var_2 = Solution_2("workouts.csv", "cities_lookup.csv")
my_var_2.L2_task_6()

,user_id,date,city,exercise,body_part,weight_kg,reps,duration_min,year,month,day
0,101,2025-08-01,Prague,Barbell Squat,legs,80,5,40,2025,8,1
1,101,2025-08-03,New York,Bench Press,chest,60,8,35,2025,8,3
2,102,2025-08-02,London,Deadlift,back,100,3,30,2025,8,2
3,103,2025-08-02,Berlin,Pull Ups,back,0,12,20,2025,8,2
4,101,2025-08-05,Prague,Running,cardio,0,0,25,2025,8,5
5,102,2025-06-05,London,Overhead Press,shoulders,40,6,25,2025,6,5
6,103,2025-08-07,Berlin,Leg Press,legs,120,8,30,2025,8,7
7,103,2025-10-12,Jamaika,Pull Ups,legs,1,11,11,2025,10,12
